## Data generation for populating a database

The database serves an API with only one endpoint: "/net_calories?start_date=<`startdate`>&end_date=<`enddate`>" that returns a value for net calories burned on the requested date. This is calculated with a SQL query that aggregates total consumption and total expenditure. For the purpose of the demo, we are adding another data source - the treadmill  to this SQL query so that we can demonstrate a method to test this change.


Tables:
1. Food Consumption - date, quantity and food_id
2. Food details - reference data to map food_id to name, unit_of_measure and calories
3. Exercise Bike - date, average watts, and total cycling time
4. Treadmill - date, speed, incline% and total running time <- this will be the new data source we add 

In [79]:
import pandas as pd
import datetime as dt
from faker import Faker
from faker_food import FoodProvider
import random

We can use faker and the faker-food package to populate foods and measurement columns

In [80]:
from faker_food import dishes
fake = Faker()
fake.add_provider(FoodProvider)

Create the food_ref table to map food_id to food attributes

In [81]:
dishes_ref = [(idx, dish) for (idx,dish) in enumerate(dishes)]

food_ref_df = pd.DataFrame(dishes_ref)

food_ref_df.rename({0:'id', 1:'name'}, axis=1, inplace=True)

food_ref_df['kcal/unit'] = [random.randint(0,10) for i in range(len(food_ref_df))]

uoms = ["kg", "g", "ml", "tbsp", "tsp"]
food_ref_df['unit_of_measure'] = [random.choice(uoms) for i in range(len(food_ref_df))]
food_cons_df = pd.DataFrame()


Create food_cons to track daily food entries

In [82]:
food_cons_df = pd.DataFrame()

# 3 meal entries a day
date_entries = pd.date_range(start=dt.date(2022, 10,1), end=dt.date(2022, 10,30)).repeat(3)
food_cons_df['date'] = date_entries
food_cons_df['food_id'] = [random.randint(0,36) for i in range(len(food_cons_df))]
food_cons_df['qty'] = [random.randint(0, 1000)/10 for i in range(len(food_cons_df))]

Create exercise_bike table - records data to help calculate expenditure

use METs to calculate calories (https://metscalculator.com/)

In [83]:
exercise_bike_df = pd.DataFrame()

exercise_bike_df['date'] = [random.choice(date_entries.unique()) for i in range(14)]
exercise_bike_df['speed_mph'] = [random.randint(1, 300)/10 for i in range(14)]
exercise_bike_df['duration_min'] = [random.randint(10, 120) for i in range(14)]

Create treadmill table

In [84]:
treadmill_df = pd.DataFrame()

treadmill_df['date'] = [random.choice(date_entries.unique()) for i in range(14)]
treadmill_df['speed_mph'] = [random.randint(1, 300)/10 for i in range(14)]
treadmill_df['incline_%'] = [random.randint(1, 150)/10 for i in range(14)]
treadmill_df['duration_min'] = [random.randint(10, 120) for i in range(14)]

In [86]:
from sqlalchemy import create_engine

engine = create_engine("postgresql://postgres@localhost:5432")

with engine.connect() as con:
    food_cons_df.to_sql("food_cons", index=False, con=con, if_exists='replace')
    food_ref_df.to_sql("food_ref", index=False, con=con, if_exists='replace')
    exercise_bike_df.to_sql("bike", index=False, con=con, if_exists='replace')
    treadmill_df.to_sql("treadmill", index=False, con=con, if_exists='replace')
    